### API demo
this is a general notebook for demonstrating the variations available with the circuit pruner api

#### model

In [ ]:
#pick a device
device = 'cuda:0'

#pick a model, any pytorch model should do, but well load in our sparsity regularized alexnet model
from circuit_explorer.utils import load_config
config_file = '../configs/alexnet_sparse_config.py'
config = load_config(config_file)
model = config.model
model = model.to(device)

#alternative
#from torchvision import models
#model = models.vgg11(pretrained = True)

layers that can be used a as target for pruning can be identified with the line below;

In [2]:
from circuit_explorer.utils import get_layers_from_model

all_layers = get_layers_from_model(model)
all_layers.keys()

odict_keys(['', 'features', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'classifier', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6'])

all keys in the above dictionary can be used to specify a target layer for pruning

#### feature selection

In [3]:
# specify target feature with unit and layer

import torch

layer = 'features.10'   #key from 'all_layers' dictionary 
unit = 1                # single unit feature target, the 2nd dimension (0 is 1st) in the layers space

#OR UNCOMMENT BELOW
#unit = torch.rand(256) # random direction feature in layers latent space ('features.10' is 256 dimensional)  

#### dataloader
our api uses pytorch dataloaders (torch.utils.data.DataLoader) to specify images to use for pruning.
Any DataLoader should do, but we provide some useful data classes as well.

In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from circuit_explorer.data_loading import rank_image_data, single_image_data




kwargs = {'num_workers': 4, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}

dataloader = DataLoader(rank_image_data('../image_data/imagenet_2/',class_folders=True),
                        batch_size=1,
                        shuffle=False,
                        **kwargs)


##OR UNCOMMENT BELOW to score with respect to a single image, we provide a simple dataloader class
# image_file_path = '../image_data/imagenet_2/Egyptian_cat/Egyptian_cat_10034.JPEG'
# dataloader = DataLoader(single_image_data(image_file_path),
#                         batch_size=1,
#                         shuffle=False,
#                         **kwargs)


#### score
get saliency scores to target feature from dataloader

In [9]:

# #kernel-wise
from circuit_explorer.score import actgrad_kernel_score
scores = actgrad_kernel_score(model,dataloader,layer,unit)

# #filter-wise
# from circuit_explorer.score import actgrad_filter_score
# scores = actgrad_filter_score(model,dataloader,layer,unit)

# #weight-wise
# from circuit_explorer.score import snip_score
# scores = snip_score(model,dataloader,layer,unit)

    # #convert weight-wise scores to structured scores
# from circuit_explorer.score import structure_scores
# scores = structure_scores(scores, model, structure='kernels')   #structure from  ['kernels','filters']

scores

OrderedDict([('features.0',
              tensor([[ 670.3234,  937.0030,  437.8027],
                      [ 478.4530, 1037.6984,  836.9188],
                      [4123.8638, 2014.0367, 2373.9856],
                      [2009.1406, 2741.1035, 1710.7646],
                      [1757.4790,   30.2473, 1730.9763],
                      [ 705.9038, 1737.6067, 1443.0610],
                      [ 704.1943, 1768.9176,  924.6486],
                      [ 536.0389,  206.9496,  295.6774],
                      [1735.5325, 1641.6807, 1693.6099],
                      [2368.1797, 2903.5029, 2396.2319],
                      [1228.3918, 1669.0922,  748.4757],
                      [2408.5095, 1209.4617,  302.6479],
                      [1672.2002, 2351.4683, 1362.9608],
                      [1664.0659, 2723.4019, 1639.6807],
                      [2219.7793, 1781.8441,  323.4003],
                      [ 304.0377,  168.1433,  349.3595],
                      [3670.8560, 4635.2104, 3017.7471],
   

#### mask
mask low scoring parameters

In [12]:
from circuit_explorer.mask import mask_from_scores, apply_mask, setup_net_for_mask

sparsity = .1
mask = mask_from_scores(scores,sparsity = sparsity, model = model,unit=unit,target_layer=layer)
apply_mask(model,mask) #model now has a weight mask inserted

#reset mask in model to all ones
#setup_net_for_mask(model)

In [13]:
model.features[6].weight_mask

Parameter containing:
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         ...,

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         ...,

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
        